# High Value Customer Identification (Insiders)

In [1]:
import re
import sqlite3
import pandas     as pd
import numpy      as np
import seaborn    as sns

from sqlalchemy import create_engine, text

from sklearn    import cluster       as c
from sklearn    import metrics       as m
from matplotlib import pyplot        as plt
from sklearn    import preprocessing as pp

## 0.1 Helper Functions

In [2]:
def descriptive_analysis(num_attributes):
    # Tendência Central - média, mediana
    ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
    ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

    # Dispersão - std, min, max, range, skew, kurtosis
    d1 = pd.DataFrame(num_attributes.apply(np.std)).T
    d2 = pd.DataFrame(num_attributes.apply(min)).T
    d3 = pd.DataFrame(num_attributes.apply(max)).T
    d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min())).T
    d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
    d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

    # Concate
    m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
    m.columns = (['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis'])
    
    return m

## 0.2 Load Dataset

In [3]:
path = '/home/matheus/repos/loyalty-clustering-programm'

data_raw = pd.read_csv(path + '/data/raw/Ecommerce.csv', encoding='unicode_escape')
data_raw = data_raw.drop(columns=['Unnamed: 8'])

# 1.0 DESCRIÇÃO DOS DADOS

In [4]:
df1 = data_raw

## 1.1 Rename columns

In [5]:
# Columns names in snake case
cols_new = ['invoice_no', 'stock_code', 'description', 'quantity', 
            'invoice_date', 'unit_price', 'customer_id', 'country']

# Renaming columns
df1.columns = cols_new

## 1.2 Data dimensions

In [6]:
# Information about shape
print(f'Number of lines: {df1.shape[0]}')
print(f'Number of columns: {df1.shape[1]}')

Number of lines: 541909
Number of columns: 8


## 1.3 Data types

In [7]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
customer_id     float64
country          object
dtype: object

## 1.4 Check NA

In [8]:
# Sum of null values
df1.isna().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

## 1.5 Replace NA

In [9]:
# Separating null values for customer id
df_missing = df1.loc[df1['customer_id'].isna(), :]
df_not_missing = df1.loc[~df1['customer_id'].isna(), :]

### Creating Customer ID

In [10]:
# Creating reference
df_backup = pd.DataFrame(df_missing['invoice_no'].drop_duplicates())
df_backup['customer_id'] = np.arange(19000, 19000+len(df_backup),1)

# Merging dataframes
df1 = pd.merge(df1, df_backup, on='invoice_no', how='left')

# Coalesce
df1['customer_id'] = df1['customer_id_x'].combine_first(df1['customer_id_y'])

df1 = df1.drop(columns=['customer_id_x', 'customer_id_y'], axis=1)

df1.isna().sum()

invoice_no         0
stock_code         0
description     1454
quantity           0
invoice_date       0
unit_price         0
country            0
customer_id        0
dtype: int64

## 1.6 Change dtypes

In [11]:
# Changing date time
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'], format='%d-%b-%y')

# Changing customer id to integer
df1['customer_id'] = df1['customer_id'].astype(int)

## 1.7 Descripte Statistics

In [12]:
# Separating numerical and categorical attributes
num_attributes = df1.select_dtypes(include=['int64', 'float64'])
cat_attributes = df1.select_dtypes(exclude=['int64', 'float64', 'datetime64[ns]'])

### 1.7.1 Numerical Attributes

In [13]:
descriptive_analysis(num_attributes)

,attributes,min,max,range,mean,median,std,skew,kurtosis
0,quantity,-80995.00,80995.0,161990.00,9.552250,3.00,218.080957,-0.264076,119769.160031
1,unit_price,-11062.06,38970.0,50032.06,4.611114,2.08,96.759764,186.506972,59005.719097
2,customer_id,12346.00,22709.0,10363.00,16688.840453,16249.00,2911.408666,0.487449,-0.804287


In [14]:
cat_attributes.sample(10)

,invoice_no,stock_code,description,country
121614,546774,22907,PACK OF 20 NAPKINS PANTRY DESIGN,United Kingdom
321287,565142,84947,ANTIQUE SILVER TEA GLASS ENGRAVED,United Kingdom
249064,558881,23284,DOORMAT KEEP CALM AND COME IN,Germany
176920,552039,23163,REGENCY SUGAR TONGS,United Kingdom
515976,579832,21485,RETROSPOT HEART HOT WATER BOTTLE,United Kingdom
348319,567423,23113,PANTRY CHOPPING BOARD,United Kingdom
47926,540458,15056BL,EDWARDIAN PARASOL BLACK,Germany
306251,563745,22608,PENS ASSORTED FUNKY JEWELED,United Kingdom
342339,566851,22722,SET OF 6 SPICE TINS PANTRY DESIGN,United Kingdom
234773,557602,20726,LUNCH BAG WOODLAND,United Kingdom


### Country

In [15]:
cat_attributes['country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [16]:
df1[['customer_id', 'country']].drop_duplicates().groupby('country').count().reset_index().sort_values('customer_id', ascending=False).head()

,country,customer_id
36,United Kingdom,7587
14,Germany,95
13,France,90
10,EIRE,44
31,Spain,31


# 2.0 FILTRAGEM DAS VARIÁVEIS

In [17]:
df2 = df1.copy()

In [18]:
# unit_price
df2 = df2[df2['unit_price'] > 0.04]

# stock_code
df2 = df2[~df2['stock_code'].isin(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY',
                                   'DCGSSGIRL', 'PADS', 'B', 'CRUK'])]

# description
df2 = df2.drop(columns='description', axis=1)

# country
df2 = df2[~df2['country'].isin(['European Community', 'Unspecified'])]

# quantity
df2_returns = df2[df2['quantity'] < 0]
df2_purchases = df2[df2['quantity'] >= 0]

# Removing bad users
df2 = df2[~df2['customer_id'].isin([16446])]

# 3.0 FEATURE ENGINEERING

In [19]:
df3 = df2.copy()

## 3.1 Feature Creation

In [20]:
# Creating a dataframe with unique customer ids
df_ref = pd.DataFrame({'customer_id': df3['customer_id']}).drop_duplicates(ignore_index=True)

### 3.1.1 Gross Revenue

In [21]:
# Creating RFM features
# Gross Revenue (quantity * unity price)
df2_purchases['gross_revenue'] = df2_purchases['quantity']*df2_purchases['unit_price']

# Monetary
df_monetary = df2_purchases.loc[:, ['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()

# Merging dataframes
df_ref = pd.merge(df_ref, df_monetary, on='customer_id', how='left')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
dtype: int64

### 3.1.2 Recency - Day From Last Purchase

In [22]:
# Recency - Last Day Purchase for each customer
df_recency = df2_purchases[['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()

# Calculating number of days since last purchase
df_recency['recency_days'] = (df2_purchases['invoice_date'].max() - df_recency['invoice_date']).dt.days
df_recency = df_recency[['customer_id', 'recency_days']]

# Merging dataframes
df_ref = pd.merge(df_ref, df_recency, on='customer_id', how='left')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
dtype: int64

### 3.1.4 Quantity of Products Purchased

In [23]:
# Number of products purchased
# Creating dataframe with number of purchases for each customer
df_freq = df2_purchases[['customer_id',
                         'stock_code']].groupby('customer_id').count().reset_index().rename(columns={'stock_code': 'qty_products'})

# Merging dataframes
df_ref = pd.merge(df_ref, df_freq, on='customer_id', how='left')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qty_products     91
dtype: int64

### 3.1.8 Frequency

In [24]:
# Aggregating max, min, number of days and number of purchases
df_aux = df2_purchases[['customer_id',
                        'invoice_no',
                        'invoice_date']].drop_duplicates().groupby('customer_id').agg(max_ = ('invoice_date', 'max'),
                                                                                      min_ = ('invoice_date', 'min'),
                                                                                      days_ = ('invoice_date', 
                                                                                               lambda x: ((x.max() - x.min()).days)+1),
                                                                                      buy_ = ('invoice_no', 'count')
                                                                                      ).reset_index()

# Frequency
df_aux['frequency'] = df_aux[['buy_', 'days_']].apply(lambda x: (x['buy_']/x['days_'])
                                                      if x['days_'] != 0 else 0, axis=1)

# Merging
df_ref = pd.merge(df_ref, df_aux[['customer_id', 'frequency']], on='customer_id', how='left')
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qty_products     91
frequency        91
dtype: int64

### 3.1.9 Number of Returns

In [25]:
# Number of Returns
df_returns = df2_returns[['customer_id', 'quantity']].groupby('customer_id').sum().reset_index().rename(columns={'quantity': 
                                                                                                                 'qty_returns'})

df_returns['qty_returns'] = df_returns['qty_returns']*-1

# Merging dataframes
df_ref = pd.merge(df_ref, df_returns, on='customer_id', how='left')
df_ref.loc[df_ref['qty_returns'].isna(), 'qty_returns'] = 0
df_ref.isna().sum()

customer_id       0
gross_revenue    91
recency_days     91
qty_products     91
frequency        91
qty_returns       0
dtype: int64

# 4.0 ANÁLISE EXPLORATÓRIA DOS DADOS (EDA)

In [26]:
df4 = df_ref.dropna().copy()

# 5.0 DATA PREPARATION

In [27]:
cols_selected = ['customer_id', 'gross_revenue', 'recency_days', 'qty_products', 'frequency', 'qty_returns']
df5 = df_ref[cols_selected].dropna().copy()

In [28]:
# Encoding features with Min Max Scaler
mm = pp.MinMaxScaler()

df5['gross_revenue'] = mm.fit_transform(df5[['gross_revenue']])
df5['recency_days']  = mm.fit_transform(df5[['recency_days']])
df5['qty_products']  = mm.fit_transform(df5[['qty_products']])
df5['frequency']     = mm.fit_transform(df5[['frequency']])
df5['qty_returns']   = mm.fit_transform(df5[['qty_returns']])

In [29]:
df5.to_csv(path + '/src/data/data_scaled.csv', index=False)

# 7.0 HYPERPARAMETER FINE-TUNNING 

In [30]:
df7 = df5.drop(columns=['customer_id']).copy()

# 8.0 MODEL TRAINING

In [31]:
df8 = df7.copy()

## 8.1 Final Model

In [32]:
# Model definition
k = 7
kmeans = c.KMeans(init='random', n_clusters=k, n_init=300, max_iter=300, random_state=42)

# Model training
kmeans.fit(df8)

# Clustering
labels = kmeans.labels_

## 8.2 Cluster Validation

In [33]:
# WSS (Within-Cluster Sum of Squares)
print(f'WSS value: {kmeans.inertia_}')

# SS (Silhouette Score)
print('Silhouette Score: {}'.format(m.silhouette_score(df8, labels, metric='euclidean')))

WSS value: 21.30378878530814


Silhouette Score: 0.49730354526511994


# 9.0 CLUSTER ANALYSIS

In [34]:
df9 = df_ref[cols_selected].dropna().copy()
df9['cluster'] = labels

# change dtypes
df9['recency_days'] = df9['recency_days'].astype(int)
df9['qty_products'] = df9['qty_products'].astype(int)
df9['qty_returns'] = df9['qty_returns'].astype(int)

In [35]:
# Number of customers
df_cluster = df9[['customer_id', 'cluster']].groupby('cluster').count().reset_index()
df_cluster['perc_customer'] = df_cluster['customer_id']/df_cluster['customer_id'].sum()*100

# Avg Gross Revenue
df_avg_gross_revenue = df9[['gross_revenue', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_gross_revenue, how='inner', on='cluster')

# Avg Recency Days
df_avg_recency_days= df9[['recency_days', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_recency_days, how='inner', on='cluster')

# Avg Products quantity
df_avg_qty_products = df9[['qty_products', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_qty_products, how='inner', on='cluster')

# Frequency
df_avg_frequency = df9[['frequency', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_frequency, how='inner', on='cluster')

# Returns
df_avg_qty_returns = df9[['qty_returns', 'cluster']].groupby('cluster').mean().reset_index()
df_cluster = pd.merge(df_cluster, df_avg_qty_returns, how='inner', on='cluster')

df_cluster.sort_values('gross_revenue', ascending=False)

,cluster,customer_id,perc_customer,gross_revenue,recency_days,qty_products,frequency,qty_returns
1,1,12,0.210711,121075.380833,4.333333,2773.250000,0.240823,824.000000
2,2,2089,36.681299,2658.026438,16.576831,138.081379,0.254557,31.580182
4,4,1167,20.491659,1053.075561,64.710368,63.405313,0.507442,8.130249
5,5,529,9.288850,956.755350,342.926276,67.996219,0.987029,142.345936
6,6,619,10.869183,865.318045,130.050081,62.550889,0.639263,3.568659
3,3,641,11.255487,657.559984,193.840874,45.485179,0.772596,4.659906
0,0,638,11.202809,637.307163,265.747649,43.534483,0.904259,19.167712


## Clusters

* **Cluster 1:** Insiders
    - Número de customers: 12 (0,21% dos costumers)
    - Receita: 12.1075,38
    - Recência em média: 4 dias
    - Quantidade média de produtos comprados: 2773
    - Frequência de compra: 0,24
    - Quantidade média de produtos retornados: 824


* **Cluster 2:**
    - Número de customers: 2089 (36,68% dos costumers)
    - Receita: 2658,03
    - Recência em média: 17 dias
    - Quantidade média de produtos comprados: 138
    - Frequência de compra: 0,25
    - Quantidade média de produtos retornados: 32


* **Cluster 4:**
    - Número de customers: 1167 (20,49% dos costumers)
    - Receita: 1053,08
    - Recência em média: 65 dias
    - Quantidade média de produtos comprados: 63
    - Frequência de compra: 0,51
    - Quantidade média de produtos retornados: 8
    

* **Cluster 5:**
    - Número de customers: 529 (9,29% dos costumers)
    - Receita: 956,76
    - Recência em média: 342 dias
    - Quantidade média de produtos comprados: 68
    - Frequência de compra: 0,98
    - Quantidade média de produtos retornados: 142
    
    
* **Cluster 6:**
    - Número de customers: 619 (10,87% dos costumers)
    - Receita: 865,32
    - Recência em média: 130 dias
    - Quantidade média de produtos comprados: 63
    - Frequência de compra: 0,64
    - Quantidade média de produtos retornados: 4
    
    
* **Cluster 3:**
    - Número de customers: 641 (11,26% dos costumers)
    - Receita: 657,56
    - Recência em média: 194 dias
    - Quantidade média de produtos comprados: 45
    - Frequência de compra: 0,77
    - Quantidade média de produtos retornados: 5
    
    
* **Cluster 0:**
    - Número de customers: 638 (11,2% dos costumers)
    - Receita: 637,31
    - Recência em média: 266 dias
    - Quantidade média de produtos comprados: 44
    - Frequência de compra: 0,9
    - Quantidade média de produtos retornados: 19

# 11.0 DEPLOY TO PRODUCTION

In [36]:
df9.dtypes

customer_id        int64
gross_revenue    float64
recency_days       int64
qty_products       int64
frequency        float64
qty_returns        int64
cluster            int32
dtype: object

## 11.1 Insert into SQLite

In [37]:
# # Create Table
# query_create_table_insiders = """
#     CREATE TABLE insiders(
#         customer_id   INTEGER,
#         gross_revenue REAL,
#         recency_days  INTEGER,
#         qty_products  INTEGER,
#         frequency     REAL,
#         qty_returns   INTEGER,
#         cluster       INTEGER
#     )      
# """

# conn = sqlite3.connect('insiders_db.sqlite')
# conn.execute(query_create_table_insiders)
# conn.commit()
# conn.close()

# # Insert Data
# conn = create_engine('sqlite:///insiders_db.sqlite')

# df9.to_sql('insiders', con=conn, if_exists='append', index=False)

In [38]:
# # Consulting Database
# query = """
#     SELECT * FROM insiders
# """

# conn = create_engine('sqlite:///insiders_db.sqlite')

# df =pd.read_sql_query(con=conn.connect(), sql=text(query))

In [39]:
# df